# Binoxxo-Solver
---
_**Module**: HSLU - Artificial Intelligence - Search & Optimization (AISO)_  
_**Source**: Slides "Constraint programming 1 - Modelling with OR-Tools"_  
_**Author**: Adrian Kauz_

>Place X or O in the empty cells so that there are no more than two consecutive X's or O's in a row or a colum. The number of X's is the same as the number of O's in each row and column. All rows and columns are unique.  
More precisely, the rows must be different from each other and separately, the columns must be differ from each other.

![Binoxxo](images/Binoxxo.png)

## Imports

In [7]:
from ortools.constraint_solver import pywrapcp
import numpy as np

## Helper functions

In [2]:
def printSolution(currentSolution):
    print("Possible solution:")
    
    for row in range(len(currentSolution)):
        currentRow = ""
        for column in range(len(currentSolution[row])):
            if(currentSolution[row][column].Value() == 0):
                currentRow += "O "
            else:
                currentRow += "X "
            
        print(currentRow)
        
    print("")

## Solution

In [9]:
solver = pywrapcp.Solver("BinoxxoSolver")

# Binoxxo to resolve:
O = 0
X = 1
_ = ''

binoxxo = [
    [_, _, _, _, _, _, _, _],
    [O, _, _, _, _, _, _, _],
    [_, X, X, _, _, O, _, _],
    [_, _, _, _, _, _, X, X],
    [_, _, _, X, _, _, _, _],
    [_, _, _, _, _, _, _, _],
    [_, _, _, _, O, _, _, O],
    [_, _, _, X, _, _, X, _]
]

# Size of the board should be an even number. Read the rules then you know why.
boardSize = len(binoxxo)

# n x n Matrix of Decision Variables
board = [[solver.IntVar(0, 1) for _j in range(boardSize)] for _i in range(boardSize)]

# Feed solver with the Binoxxo Puzzle
for row in range(boardSize):
    for column in range(boardSize):
        if (binoxxo[row][column] != ''):
            solver.Add(board[row][column] == binoxxo[row][column])


# --------------------------------------------------------------------------------
# Set constraints
# --------------------------------------------------------------------------------
# Each row and each column should have an equal amount of X and O's:
for i in range(boardSize):
    solver.Add(solver.Sum([board[i][j] for j in range(boardSize)]) == int(boardSize / 2)) # Row
    solver.Add(solver.Sum([board[j][i] for j in range(boardSize)]) == int(boardSize / 2)) # Column

# Each row and column should be different. Hint: Think in binary-patterns and their representation.
for i in range(boardSize):
    for x in range(i + 1, boardSize):
        solver.Add(solver.Sum([board[i][j] * 2**j for j in range(boardSize)]) !=
                   solver.Sum([board[x][j] * 2**j for j in range(boardSize)]))
    
        solver.Add(solver.Sum([board[j][i] * 2**j for j in range(boardSize)]) !=
                   solver.Sum([board[j][x] * 2**j for j in range(boardSize)]))

# Max two cells next to each other should have the same value
for i in range(boardSize):
    for j in range(boardSize - 2):
        # Rows
        solver.Add(solver.Sum([board[i][j + x] for x in range(3)]) > 0)
        solver.Add(solver.Sum([board[i][j + x] for x in range(3)]) < 3)
    
        # Columns
        solver.Add(solver.Sum([board[j + x][i] for x in range(3)]) > 0)
        solver.Add(solver.Sum([board[j + x][i] for x in range(3)]) < 3)


# --------------------------------------------------------------------------------
# Now solve it!
# --------------------------------------------------------------------------------
db = solver.Phase(list(np.concatenate(board)), # Sudoku board as array
                solver.INT_VAR_SIMPLE,         # Variable selection policy for search
                solver.INT_VALUE_SIMPLE)       # Value selection policy for search

solver.NewSearch(db)

if solver.NextSolution():
    printSolution(board)
    
solver.EndSearch()

#Display the number of solutions found:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime: {}ms".format(solver.WallTime()))
print("Failures: {}".format(solver.Failures()))
print("Branches: {}".format(solver.Branches()))

Possible solution:
X O O X O X O X 
O O X O X X O X 
O X X O X O X O 
X O O X O O X X 
X X O X O X O O 
O X X O X O O X 
X O X O O X X O 
O X O X X O X O 

Solutions: 1
Runtime: 11ms
Failures: 9
Branches: 24
